<a href="https://colab.research.google.com/github/aerimili/EIE401-1-Multimedia/blob/main/Actividades/Actividad_5/Actividad_5_VillalobosAlejandra_20914803_K.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center>

</center></h1>
<font size="6"><center>
EIE 401
PROCESAMIENTO DIGITAL MULTIMEDIA
</center></font>


<center><h2>Actividad 5</h2></center>
<center><h3>Audio</h3></center>
<center><h3>Profesor: Jorge Cardenas</h3></center>

<center><h3>Por: Alejandra Villalobos</h3></center>
<center><h5>Pontificia Universidad Catolica de Valparaiso</h5></center>


## 1. Aplique la transformada rápida de fourier a una señal de audio
<p align="justify">
El archivo de audio debe ser formato .WAV, con menos de 30 segundo de duración.


In [70]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy import signal
from IPython.display import Audio

In [ ]:
from google.colab import files

#Subir archivo de audio
print("Cargue archivo de audio: ")
uploaded = files.upload()

In [72]:
ruta = "Exercise2.7_adultmale_b_d2.wav" #Definición de ruta del audio

fs, datos = wavfile.read(ruta) #Leer archivo de audio y extraer la frecuencia de muestreo y el audio
datos = datos.astype(np.float32)  #Converción de audio a valores float32

t = len(datos) / fs #Definición de duración de audio
tiempo = np.linspace(0, t, len(datos)) #Definición de vector tiempo

In [ ]:
#Gráfica de señal de audio original
plt.figure(figsize = (20,6))
plt.plot(tiempo, datos)
plt.title("Señal de audio")
plt.xlabel("Tiempo(s)")
plt.ylabel("Amplitud")

In [74]:
transformada = np.fft.fft(datos) #Aplicación de transformada de Fourier de señal de audio
frecuencias = np.fft.fftfreq(len(datos), 1/fs) #Calculo de componentes frecuenciales

In [ ]:
#Gráfica de espectro de frecuencia de la señal
plt.figure(figsize = (12,5))
plt.plot(np.abs(frecuencias), np.abs(transformada))
plt.title("Espectro de frecuencia de señal de audio.")
plt.xlabel("Frecuencia(Hz)")
plt.ylabel("Magnitud")

In [ ]:
Audio(datos, rate = fs) #Reproducción de señal de audio original

## 2. Obtén el espectrograma de tu señal de audio

In [41]:
!pip install librosa #Instalación de librería librosa

In [77]:
import librosa

X = librosa.stft(datos) #Transformada de Fourier de tiempo corto de señal

xdb = librosa.amplitude_to_db(abs(X)) #Conversión de transformada a decibeles

In [ ]:
#Visualización del espectrograma
librosa.display.specshow(xdb, sr = fs, x_axis = "time", y_axis="log")
plt.title("Espectrograma de señal de audio")
plt.colorbar()

El espectrograma muestra cómo varía la energía de la señal de audio en función del tiempo y la frecuencia. Cada palabra pronunciada en el audio se representa como una banda vertical con mayor energía en frecuencias más bajas. A frecuencias más altas se puede ver una disminución gradual de la energía.

## 3. Slicing
Desarrolla un framento de código que te permita tomar una señal de audio y framentarla en N segmentos o slices. Debe asignar la longitud temporal de cada slice y el programa debe indicar el número de slices que quedan. Considera que al final puede quedar un slice de longitud diferente a todos los demas.

En el slicing debes también incluir un gap o overlap a lado y lado de cada uno de los fragmentos. Este overlap debe ser ajustable del número de muestras que constituyen dicho overlap.


In [ ]:
def slicing(datos, tiempoSlice, overlap):
  """
  Realizar fragmentación de señal de audio con overlap.

  Parámetros:
  datos: Señal de audio
  tiempo_slice: Longitud temporal de cada slice
  overlap: Solapamiento entre cada fragmento
  """

  nMuestras = len(datos) #Definición de número de muestras de señal de audio
  tiempoTotal = nMuestras/fs #Tiempo total de la señal de audio
  nSlices = int(tiempoTotal/tiempoSlice) #Número de slices redondeado
  nSlicesReal = tiempoTotal / tiempoSlice #Número de slices sin redondeo
  slices = [] #Lista vacía para almacenar los slices generados
  muestrasSlice = int(tiempoSlice * fs) #Número de muestras de cada slice
  muestrasOverlap = int(overlap * fs) #Número de muestras de overlap

  #Generación de slices con overlap
  for i in range(nSlices):
    inicio = int(i * (tiempoSlice * fs - overlap))
    fin = inicio + muestrasSlice
    slices.append(datos[inicio:fin])

  print(f"Se generaron {nSlices} slices.") #Imprimir en pantalla el número de slices que se generaron

  print(f"Cada slice dura {tiempoSlice} segundos.") #Imprime en pantalla la longitud temporal de cada slice

  #Si hay un último slice de diferente tamaño, calcula e imprime en pantalla que existe y su duración
  if fin < nMuestras:
    ultimoInicio = nMuestras - muestrasSlice
    ultimoSlice = datos[ultimoInicio:]
    slices.append(ultimoSlice)
    print("Se genero un ultimo slice con longitud diferente.")

    tiempoUltimoSlice = (nSlicesReal - nSlices) * tiempoSlice
    print(f"El ultimo slice dura {round(tiempoUltimoSlice, 3)} segundos.")

  return slices

#Definir parámetros de la función
tiempoSlice = 0.5 #Longitud temporal de cada slice
overlap = 1600 #Número de muestras de overlap

audioSlicing = slicing(datos, tiempoSlice, overlap) #Llama a la función para generar slices


## 3.1 Slicing usando Pytorch de la señal obtenida en el primer punto.
Debes crear un array con los slices creados a partir de tensores en pytorch.
Grafica un par de tus slices y reproduce el audio.


In [ ]:
import torch

datosTensor = torch.tensor(datos) #Convertir valores de la señal de audio en tensor

def slicing_torch(datosTorch, tiempoSliceTorch, overlapTorch):
  """
  Realizar fragmentación de señal de audio con overlap usando pytorch.

  Parámetros:
  datos_torch: Señal de audio en tensor
  tiempo_slice: Longitud temporal de cada slice
  overlap: Solapamiento entre cada fragmento
  """

  nMuestrasTorch = len(datosTorch) #Definición de número de muestras de señal de audio
  tiempoTotalTorch = nMuestrasTorch/fs #Tiempo total de la señal de audio
  nSlicesTorch = int(tiempoTotalTorch/tiempoSliceTorch) #Número de slices redondeado
  nSlicesRealTorch = tiempoTotalTorch / tiempoSliceTorch #Número de slices sin redondeo
  slicesTorch = [] #Lista vacía para almacenar los slices generados
  muestrasSliceTorch = int(tiempoSliceTorch * fs) #Número de muestras de cada slice
  muestrasOverlapTorch = overlapTorch * fs #Número de muestras de overlap

  #Generación de slices con overlap
  for i in range(nSlicesTorch):
    inicioTorch = int(i * (tiempoSliceTorch *fs - overlapTorch))
    finTorch = inicioTorch + muestrasSliceTorch
    slicesTorch.append(datos[inicioTorch:finTorch])

  print(f"Se generaron {nSlicesTorch} slices.") #Imprimir en pantalla el número de slices que se generaron

  print(f"Cada slice dura {tiempoSliceTorch} segundos.") #Imprime en pantalla la longitud temporal de cada slice

  #Si hay un último slice de diferente tamaño, calcula e imprime en pantalla que existe y su duración
  if finTorch < nMuestrasTorch:
    ultimoInicioTorch = nMuestrasTorch - muestrasSliceTorch
    ultimoSliceTorch = datos[ultimoInicioTorch:]
    slicesTorch.append(torch.tensor(ultimoSliceTorch))
    print("Se genero un ultimo slice con longitud diferente.")

    tiempoUltimoSliceTorch = (nSlicesRealTorch - nSlicesTorch) * tiempoSliceTorch
    print(f"El ultimo slice dura {round(tiempoUltimoSliceTorch, 3)} segundos.")

  return slicesTorch

#Definir parámetros de la función
tiempoSliceTorch = 2 #Longitud temporal de cada slice
overlapTorch = 20000 #Número de muestras de overlap

audioSlicingTorch = slicing_torch(datosTensor, tiempoSliceTorch, overlapTorch) #Llama a la función para generar slices

In [ ]:
time = torch.arange(0, len(audioSlicingTorch[0])) / fs #Vector de tiempo del primer slice
time2 = torch.arange(len(audioSlicingTorch[0]) - int(overlapTorch), len(audioSlicingTorch[0]) + len(audioSlicingTorch[1]) - int(overlapTorch)) / fs #Vectorde tiempo de 2do slice

#Gráfica de primer y segundo slice de la señal de audio
plt.figure(figsize=(12, 8))
plt.subplot(2,1,1)
plt.plot(time, audioSlicingTorch[0])
plt.title('Primer slice')
plt.xlabel("Tiempo(s)")
plt.ylabel("Amplitud")
plt.subplot(2,1,2)
plt.plot(time2, audioSlicingTorch[1])
plt.title('Segundo slice')
plt.xlabel("Tiempo(s)")
plt.ylabel("Amplitud")

In [ ]:
Audio(audioSlicingTorch[0], rate = fs) #Reproducción de primer slice

In [ ]:
Audio(audioSlicingTorch[1], rate = fs) #Reproducción de segundo slice

## 4. Filtro Butterworth para una señal de audio.
Diseña y aplica un filtro butterworth a tu señal de audio.
Muestra un gráfico con la respuesta del filtro butterworth y explica el resultado.

In [ ]:
ordenes = [8, 6, 4, 2] #Lista de ordenes a comparar

datosFiltrado = [] #Lista vacía para almacenar datos de señal filtrada

for orden in ordenes:

  b, a = signal.butter(orden, 1000, fs = fs, btype = "low") #Diseño de filtro Butterworth

  w, h = signal.freqz(b, a, fs=fs) #Cálculo de respuesta en frecuencia del filtro
  plt.plot(w, 20*np.log10(abs(h)), label = f"Orden {orden}") #Gráfica de respuesta en frecuencia de señal filtrada

  dataFiltrada = signal.lfilter(b, a, datos) #Filtrar la señal
  datosFiltrado.append(dataFiltrada)  #Agregar datos filtrados a la lista

#Agregar características de gráfico
plt.title('Respuesta de los filtros Butterworth en distintos oredenes')
plt.xlabel('Frecuencia [Hz]')
plt.ylabel('Ganancia [dB]')
plt.legend()

La ganancia negativa en el gráfico indica que el filtro está atenuando la señal, especialmente para eliminar el ruido y conservar únicamente la información relevante. Se observa que a medida que aumenta el orden del filtro, la caída de la ganancia se vuelve más pronunciada, lo que se traduce en una pendiente más empinada en la respuesta del filtro. Cada curva en el gráfico representa cómo el filtro atenúa gradualmente las frecuencias por encima de la frecuencia de corte.

In [ ]:
#Gráfica de señal de audio original v/s señal de audio filtrada
plt.figure(figsize = (20,6))
plt.plot(tiempo, datos, label = "Señal original")
plt.plot(tiempo, datosFiltrado[3], label = "Señal filtrada")
plt.title("Comparacion de audio original y audio filtrado con orden 2")
plt.xlabel("Tiempo(s)")
plt.ylabel("Amplitud")
plt.legend()

In [ ]:
Audio(datosFiltrado[3], rate = fs) #Reproducción de señal de audio filtrada

## 4. Referencias
<p align="justify">
    
[Análisis espectral para audio] https://ccrma.stanford.edu/~jos/mdft/mdft-python.html

[Análisis espectral] https://currents.soest.hawaii.edu/ocn_data_analysis/_static/Spectrum.html

[FFT en audio] https://realpython.com/python-scipy-fft/#time-domain-vs-frequency-domain

[Filtro] https://pythonguia.com/filtro-de-python-scipy-butterworth/